In [6]:
import pandas as pd
from pathlib import Path
from glob import glob

In [7]:
cwd = Path.cwd()
dp = cwd.parent / 'data'
output = cwd.parent / 'output'
files = glob(str(dp / '**/API*.csv'))

In [8]:
# Shows the sum of NaN values in each column for years 2000-2021
# 2019 is the most recent year with the least NaN values
for year in range(2021, 2000, -1):

    year = str(year)
    sf = files[0]

    tdf = pd.read_csv(sf, skiprows=4)
    tdf = tdf.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values=year)

    for file in files[1:]:
        df = pd.read_csv(file, skiprows=4)
        pivot = df.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values=year)
        tdf = tdf.merge(pivot, on=['Country Name', 'Country Code'], how='inner')
    
    print(year, ':',  tdf.isna().sum().sum())
    print(tdf.isna().sum())

2021 : 1332
Indicator Code
EG.CFT.ACCS.ZS    266
EG.ELC.ACCS.ZS    266
EG.FEC.RNEW.ZS    266
EN.ATM.CO2E.PC    266
SP.DYN.LE00.IN    266
SP.POP.TOTL         2
dtype: int64
2020 : 585
Indicator Code
EG.CFT.ACCS.ZS     29
EG.ELC.ACCS.ZS      3
EG.FEC.RNEW.ZS    266
EN.ATM.CO2E.PC    266
SP.DYN.LE00.IN     19
SP.POP.TOTL         2
dtype: int64
2019 : 132
Indicator Code
EG.CFT.ACCS.ZS    29
EG.ELC.ACCS.ZS     2
EG.FEC.RNEW.ZS    53
EN.ATM.CO2E.PC    27
SP.DYN.LE00.IN    19
SP.POP.TOTL        2
dtype: int64
2018 : 133
Indicator Code
EG.CFT.ACCS.ZS    29
EG.ELC.ACCS.ZS     3
EG.FEC.RNEW.ZS    53
EN.ATM.CO2E.PC    27
SP.DYN.LE00.IN    19
SP.POP.TOTL        2
dtype: int64
2017 : 131
Indicator Code
EG.CFT.ACCS.ZS    29
EG.ELC.ACCS.ZS     2
EG.FEC.RNEW.ZS    53
EN.ATM.CO2E.PC    27
SP.DYN.LE00.IN    18
SP.POP.TOTL        2
dtype: int64
2016 : 129
Indicator Code
EG.CFT.ACCS.ZS    28
EG.ELC.ACCS.ZS     2
EG.FEC.RNEW.ZS    53
EN.ATM.CO2E.PC    27
SP.DYN.LE00.IN    17
SP.POP.TOTL        2
dtype: int

In [9]:
# Joins all the data into one dataframe for year 2019

sf = files[0]

tdf = pd.read_csv(sf, skiprows=4)
tdf = tdf.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values='2019')

for file in files[1:]:
    df = pd.read_csv(file, skiprows=4)
    pivot = df.pivot(index=['Country Name', 'Country Code'], columns='Indicator Code', values='2019')
    tdf = tdf.merge(pivot, on=['Country Name', 'Country Code'], how='inner')

In [10]:
tdf = tdf.reset_index()
tdf = tdf.dropna()

In [11]:
tdf

Indicator Code,Country Name,Country Code,EG.CFT.ACCS.ZS,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EN.ATM.CO2E.PC,SP.DYN.LE00.IN,SP.POP.TOTL
0,Afghanistan,AFG,31.9,97.699997,18.51,0.159824,64.833,38041757.0
3,Albania,ALB,80.7,100.000000,40.20,1.692248,78.573,2854191.0
4,Algeria,DZA,99.7,99.500000,0.16,3.977650,76.880,43053054.0
7,Angola,AGO,49.5,45.642799,54.69,0.792137,61.147,31825299.0
8,Antigua and Barbuda,ATG,100.0,100.000000,0.91,5.354476,77.016,97115.0
...,...,...,...,...,...,...,...,...
258,"Venezuela, RB",VEN,97.0,99.976814,15.62,3.939566,72.064,28515829.0
259,Vietnam,VNM,64.4,99.400002,18.65,3.488313,75.400,96462108.0
263,"Yemen, Rep.",YEM,61.3,72.751076,3.11,0.380633,66.125,29161922.0
264,Zambia,ZMB,11.2,43.000000,84.50,0.380717,63.886,17861034.0


In [12]:
tdf.to_csv(output / 'joined_data.csv', index=False)